In [1]:
import numpy as np
import nevergrad as ng
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

# Functions to be tested

For now, use a very simple function - quadratic with max value 0, and theta* = [0.5, 0.5] 

Also note that we will maximize the function, not minimize.

In [2]:
def F(theta):
    if theta.ndim == 1:
        theta = np.expand_dims(theta, 0)
    return -np.sum((theta - 0.5) ** 2, axis=tuple(range(theta.ndim)[1:]))

# Hessian Estimator


In [17]:
def hessian_estimator_ours(F, theta, samples, sigma):
    """
    Input:
        F: function
        theta: point of interest, should be 1-d array of length d
        samples: number of samples
        sigma: smoothing parameter
    """
    d = len(theta)
    eps = np.random.normal(size=d*samples).reshape((samples, d))
    Fs = F(theta+sigma*eps)
    eps = np.expand_dims(eps, -1)
    return (
        (eps*np.transpose(eps, (0, 2, 1)) - np.identity(d))
        * Fs.reshape(-1, 1, 1)
    )/(sigma**2)
def hessian_estimator_HessianAware(F, theta, samples, sigma):
    """
    Input:
        F: function
        theta: point of interest, should be 1-d array of length d
        samples: number of samples
        sigma: smoothing parameter
    """
    d = len(theta)
    eps = np.random.normal(size=d*samples).reshape((samples, d))
    Fs = (F(theta+sigma*eps)+F(theta-sigma*eps))/2-F(theta)
    eps = np.expand_dims(eps, -1)
    return (
        (eps*np.transpose(eps, (0, 2, 1)))
        * Fs.reshape(-1, 1, 1)
    )/(sigma**2)

In [73]:
theta = np.array([0.5,0.5])
samples = 1000000
sigma = 0.01
Htrue = np.array([[-2, 0], [0, -2]])

In [80]:
# %%timeit
Hsamples = hessian_estimator_ours(F, theta, samples, sigma)
print("Average:")
print(Hsamples.mean(axis=0))
print("MSE:")
print(sum((Hsamples - Htrue)**2)/samples)

Average:
[[-2.01388467  0.00516316]
 [ 0.00516316 -1.99888951]]
MSE:
[[102.34559342  48.75876572]
 [ 48.75876572 100.67647477]]


In [81]:
# %%timeit
Hsamples = hessian_estimator_HessianAware(F, theta, samples, sigma)
print("Average:")
print(Hsamples.mean(axis=0))
print("MSE:")
print(sum((Hsamples - Htrue)**2)/samples)

Average:
[[-3.99239995e+00  2.66210284e-03]
 [ 2.66210284e-03 -4.00130918e+00]]
MSE:
[[132.10182507  47.65126161]
 [ 47.65126161 132.95327533]]
